## Core Functions
These handle importing necessary libraries, preparation of the feature arrays for Machine Learning, and execution of Machine Learning training and testing.

In [1]:
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from Scripts_Python.ML_Python_Build_FeatureArrays_FromROOT import Build_FeatureArrays_FromROOT
from Scripts_Python.ML_Python_TrainTest import (
    Build_FeatureArrays_FromCSV,
    Write_MLResults_ToCSV,
    Write_MLWeights_ToCSV,
    Train_All_Estimators,
    Train_LinearRegression,
    Train_RandomForestRegression,
    Train_MLPRegression,
    Test_Estimator,
    Test_All_Estimators)
from datetime import datetime



def Build_SelectFeatureArray(
    X_features,
    feature_index
    ) :
    """
    Builds training and testing data sets
    """
    
    print("Selecting data from master array...")
    
    X_features_select = []
    for i in range(len(X_features)):
        X_temp = []
        for j in range(len(feature_index)):
            X_temp.append(X_features[i][feature_index[j]])
        X_features_select.append(X_temp)
        
    print("Data ready. Feature array length:", len(X_features_select), "\n")
    
    return X_features_select

    

def TestAndSave_LinearRegression(
    feature_label,    # Array of labels corresponding to each feature
    feature_index,    # Array of indices for each feature used in X_train
    lr_pipeline,      # Trained Linear Regression Pipeline
    lr_coeffs,        # Array of coefficient values from trained linear regression pipeline
    X_test_select,    # Array of testing data features
    y_test,           # Array of testing data targets
    sc_test,          # Array of testing data simple corrections
    pt_test_min,      # Float of min pT to test with
    pt_test_max,      # Float of max pT to test with
    output_filename,  # Directory path + name for output csv file
    use_scaler = True # If true, rescales data
    ) :
    
    X_test_temp  = []
    y_test_temp  = []
    sc_test_temp = []
    
    for i in range(len(y_test)):
        if y_test[i] > pt_test_min and y_test[i] < pt_test_max:
            X_test_temp.append(X_test_select[i])
            y_test_temp.append(y_test[i])
            sc_test_temp.append(sc_test[i])
        else: continue
    
    # Tests estimator
    
    print(type(lr_pipeline))
    
    lr_results, lr_results_delta = Test_Estimator(
        lr_pipeline,
        X_test_temp, 
        y_test_temp
        )
    
    # Writes outputs to a csv file
    Write_MLResults_ToCSV(
        output_filename,
        y_test_temp,
        sc_test_temp,
        lr_results,
        X_test_temp,
        feature_label
        )
    
    return



def TrainTestPlot_All_Estimators(
    feature_label,    # Array of labels corresponding to each feature
    feature_index,    # Array of indices for each feature used in X_train
    X_train,          # Array of training data features
    y_train,          # Array of training data targets
    sc_train,         # Array of training data simple correction values
    X_test,           # Array of testing data features
    y_test,           # Array of testing data targets
    sc_test,          # Array of testing data simple corrections
    output_file_path, # File path for outputs
    use_scaler = True,
    use_lr = True,
    use_rf = True,
    use_mlp = True,
    ) :
    
    # Builds training data set
    print("Selecting training data...")
    X_train_select = []
    for i in range(len(X_train)):
        X_temp = []
        for j in range(len(feature_index)):
            X_temp.append(X_train[i][feature_index[j]])
        X_train_select.append(X_temp)
    print("Training data ready. X/Y length:", len(X_train_select), len(y_train), "/n")
    
    # Builds pipelines from selected training features
    print("Building estimator pipelines...")
    lr_pipeline, rf_pipeline, mlp_pipeline, lr_coeffs, rf_features = Train_All_Estimators(
        X_train_select, y_train, feature_label, 
        use_StandardScaler = use_scaler,
        use_LinearRegression = use_lr,
        use_RandomForest = use_rf,
        use_MLP = use_mlp)
    print("Pipelines built./n")
    
    print("Selecting testing data...")
    X_test_select = []
    for i in range(len(X_test)):
        X_temp = []
        for j in range(len(feature_index)):
            X_temp.append(X_test[i][feature_index[j]])
        X_test_select.append(X_temp)
    print("Testing data ready. X/Y length:", len(X_test_select), len(y_test), "/n")
    
    # Test estimators
    print("Testing all estimators...")
    lr_results, lr_results_delta, rf_results, rf_results_delta, mlp_results, mlp_results_delta = Test_All_Estimators(
        X_test_select, 
        y_test, 
        lr_pipeline,
        rf_pipeline,
        mlp_pipeline)
    print("Estimator testing complete!/n")
    
    return



###########################################
#                                         #
#     DATA PREPARATION - CHANGE BELOW     #
#                                         #
###########################################



file_directory   = "../../Files/Comparison_Test_4/Data/"

train_base_name  = "Train_B8_10_90_N500000" # Cut off 'ML_Prep_' and '.root' parts of input file names
train_bias       = "B8"
train_range      = (10., 90.) # pT min/max of training file

test_base_name   = "Test_B8_10_90_N500000" # Cut off 'ML_Prep_' and '.root' parts of input file names
test_range       = (10., 90.) # pT min/max of testing file



##### ANYTHING BELOW THIS SHOULDN'T NEED TO CHANGE #####

train_file_name  = "Full_" + train_base_name + "_ML_Prep.root"
# train_file_name  = "Full_Train_TopInRange_B8_10_90_N500000_ML_Prep.root"
train_tree_name  = "ML_" + train_base_name
train_file_path  = file_directory + train_file_name
train_csv_path   = file_directory + "ML_Prep_" + train_base_name + "_Backup.csv"
# train_csv_path   = file_directory + "ML_Prep_Train_TopInRange_B8_10_90_N500000_Backup.csv"

test_file_name   = "Full_" + test_base_name + "_ML_Prep.root"
test_tree_name   = "ML_" + test_base_name
test_file_path   = file_directory + test_file_name
test_csv_path    = file_directory + "ML_Prep_" + test_base_name + "_Backup.csv"

# Builds ML output directories
output_directory = file_directory + "ML_Results/"

try:
    os.mkdir(output_directory)
    print("made 'ML_Results' directory")
except:
    print("directory already exists")

# Rebuilds feature and target arrays from csv file, or rebuilds them if csv doesn't exist

# Training data
if os.path.exists(train_csv_path):
    X_train, y_train, sc_train = Build_FeatureArrays_FromCSV(train_csv_path)
else:
    X_train, y_train, sc_train = Build_FeatureArrays_FromROOT(
        train_file_path, train_tree_name, train_csv_path, train_range[0], train_range[1])

# Testing data
if os.path.exists(test_csv_path):
    X_test,  y_test,  sc_test  = Build_FeatureArrays_FromCSV(test_csv_path)
else:
    X_test, y_test, sc_test = Build_FeatureArrays_FromROOT(
        test_file_path,  test_tree_name,  test_csv_path,  test_range[0],  test_range[1])

# Set Features to train with
# X_values[
#    0  jet_pt_raw,      1  jet_pt_corr,     2  jet_mass,        3  jet_area, 
#    4  jet_area_err,    5  jet_const_n,     6  const_pt_mean,   7  const_pt_median, 
#    8  const_1_pt,      9  const_2_pt,      10 const_3_pt,      11 const_4_pt,
#    12 const_5_pt,      13 const_6_pt,      14 const_7_pt,      15 const_8_pt,
#    16 const_9_pt,      17 const_10_pt,     18 jet_y,           19 jet_phi,
#    20 jet_rho]

# Training with 1 feature
feature_label_1feat = [
    "jet_pt_raw"]
feature_index_1feat = [0]

# Training with 3 features
feature_label_3feat = [
    "jet_pt_raw", "jet_area", "jet_rho"]
feature_index_3feat = [0, 3, 20]

# Training with 12 features
feature_label_12feat = [
    "jet_pt_raw",    "jet_pt_corr",    "jet_mass",      "jet_area", 
    "jet_const_n",   "const_pt_mean",  "const_1_pt",    "const_2_pt",
    "const_3_pt",    "const_4_pt",     "jet_y",         "jet_rho"]
feature_index_12feat = [0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 18, 20]



now = datetime.now()
dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
print("\nReady!", dt_string)

Welcome to JupyROOT 6.26/04
made 'ML_Results' directory
Input file accessed successfully. Output file generated.
Accessing input tree...
Input tree accessed successfully.
Creating .csv backup file...
Backup file started.
Preparing to collect data from TTree...
Jet: 10000 | pTraw: 60.188 | pTcorr:  8.440 | pTtrue:  14.256
Jet: 20000 | pTraw: 32.027 | pTcorr: -14.332 | pTtrue:  12.513
Jet: 30000 | pTraw: 143.847 | pTcorr:  90.437 | pTtrue:  83.539
Jet: 40000 | pTraw: 106.050 | pTcorr:  41.896 | pTtrue:  38.357
Jet: 50000 | pTraw: 110.317 | pTcorr:  51.624 | pTtrue:  49.293
Jet: 60000 | pTraw: 80.943 | pTcorr:  35.673 | pTtrue:  21.372
Jet: 70000 | pTraw: 55.794 | pTcorr:  17.520 | pTtrue:  12.432
Jet: 80000 | pTraw: 80.902 | pTcorr:  40.693 | pTtrue:  40.140
Jet: 90000 | pTraw: 87.102 | pTcorr:  33.998 | pTtrue:  34.437
Jet: 100000 | pTraw: 76.148 | pTcorr:  15.593 | pTtrue:  25.924
Jet: 110000 | pTraw: 81.343 | pTcorr:  13.017 | pTtrue:  12.357
Jet: 120000 | pTraw: 104.953 | pTcorr:  46

Jet: 540000 | pTraw: 43.387 | pTcorr: -10.636 | pTtrue:  12.306
Jet: 550000 | pTraw: 87.387 | pTcorr:  32.127 | pTtrue:  15.246
Jet: 560000 | pTraw: 65.468 | pTcorr:  16.474 | pTtrue:  22.560
Jet: 570000 | pTraw: 57.056 | pTcorr:  9.328 | pTtrue:  10.338
Jet: 580000 | pTraw: 133.598 | pTcorr:  77.512 | pTtrue:  73.817
Jet: 590000 | pTraw: 86.909 | pTcorr:  34.983 | pTtrue:  23.604
Jet: 600000 | pTraw: 49.410 | pTcorr:  8.561 | pTtrue:  10.564
Jet: 610000 | pTraw: 93.441 | pTcorr:  39.968 | pTtrue:  45.023
Jet: 620000 | pTraw: 70.647 | pTcorr:  16.035 | pTtrue:  37.602
Jet: 630000 | pTraw: 111.472 | pTcorr:  52.279 | pTtrue:  60.204
Jet: 640000 | pTraw: 114.735 | pTcorr:  67.078 | pTtrue:  74.832
Jet: 650000 | pTraw: 109.793 | pTcorr:  43.529 | pTtrue:  21.578
Jet: 660000 | pTraw: 98.687 | pTcorr:  35.622 | pTtrue:  33.038
Backup .csv file closed.
All data transferred to array. Testing with 661566 jets.

Data set lengths: 661566 / 661566 / 661566
Input file closed.

Ready! 2023/01/26 13

## Training & Testing
1 Feature: pt_raw ONLY

3 Features: pt_raw, jet_area, jet_rho

12 Features: jet_pt_raw, jet_pt_corr, jet_mass, jet_area, jet_const_n, const_pt_mean, const_1_pt, const_2_pt, const_3_pt, const_4_pt, jet_y, jet_rho

In [2]:
#################################################
#                                               #
#  TRAIN ML ON ONE BIN, WITH 1, 3, 12 FEATURES  #
#                                               #
#################################################

# Builds outputs directories
output_directory_alt = output_directory + "/Test_4GeV_Bins/"

try:
    os.mkdir(output_directory_alt)
    print("made directory")
except:
    print("directory already exists")
try:
    os.mkdir(output_directory_alt + "Plots_Actual/")
    os.mkdir(output_directory_alt + "Plots_Delta/")
    print("made subdirectories")
except:
    print("directory already exists")

test_min_max_array = [  # Array of min and max for pT ranges to test on
    [18,22], [28,32], [38,42], [48,52], 
    [58,62], [68,72], [78,82], [40,60]
    ]
feature_bundle = [
    [feature_label_1feat,  feature_index_1feat], 
    [feature_label_3feat,  feature_index_3feat],
    [feature_label_12feat, feature_index_12feat]
    ]
train_bundle = [ # This may be implemented later to iterate through multiple training sets
    [X_train, y_train, sc_train]
    ]

for feature_set in feature_bundle:
    feature_label = feature_set[0]
    feature_coeff_label = feature_set[0].copy()
    feature_index = feature_set[1]
    
    output_csv_name = output_directory_alt + "Train_" + train_bias + "_F" + str(len(feature_label)) + "_" + str(int(train_range[0])) + "_" + str(int(train_range[1]))
    
    feature_coeff_label.append("lr_intercept") # Adds field for linear regression y-intercept
    
    # Builds training and testing arrays
    print("\nBuilding training and testing selected feature arrays...")
    X_train_select = Build_SelectFeatureArray(X_train, feature_index)
    X_test_select  = Build_SelectFeatureArray(X_test, feature_index)

    # Trains estimator
    print("\nTraining linear regression estimator...")
    lr_pipeline, lr_coeffs = Train_LinearRegression(
        X_train_select, 
        y_train, 
        feature_coeff_label, 
        use_scaler = True)
    print(type(lr_pipeline))
    
    Write_MLWeights_ToCSV(
        output_csv_name + "_LR_Coeffs.csv",
        lr_coeffs, 
        feature_coeff_label
        )

    # Tests estimator and saves results

    for min_max in test_min_max_array:
        
        output = "\nTesting " + str(len(feature_index)) + " features on " + str(min_max[0]) + "-" + str(min_max[1]) + " GeV..."
        print(output)
        
        csv_path = output_csv_name + "_Test_" + str(int(min_max[0])) + "_" + str(int(min_max[1])) + ".csv"
        
        TestAndSave_LinearRegression(
            feature_label,
            feature_index, 
            lr_pipeline, 
            lr_coeffs,
            X_test_select,
            y_test, 
            sc_test,
            min_max[0],
            min_max[1],   
            csv_path,
            use_scaler = True
            )
        
        print("Test and save complete!\n")



#################################################
#                                               #
#  TRAIN ML OVER 20 GeV BINS, USES 12 FEATURES  #
#                                               #
#################################################

# Builds outputs directories
output_directory_alt = output_directory + "Train_20GeV_Bins/"

try:
    os.mkdir(output_directory_alt)
    print("made directory")
except:
    print("directory already exists")
try:
    os.mkdir(output_directory_alt + "Plots_Actual/")
    os.mkdir(output_directory_alt + "Plots_Delta/")
    print("made subdirectories")
except:
    print("directory already exists")

training_bundle = [
    [10.,30.], [20.,40.], [30.,50.], [40.,60.],
    [50.,70.], [60.,80.], [70.,90.]
    ]

# ONLY runs 12 features
for training_range in training_bundle:
    train_min = training_range[0]
    train_max = training_range[1]
    
    output_csv_name = output_directory_alt + "Train_" + train_bias + "_F12_" + str(int(train_min)) + "_" + str(int(train_max))
    
    X_train_cut = []
    y_train_cut = []
    sc_train_cut = []
    
    for i in range(len(X_train)):
        if (y_train[i] > train_min) and (y_train[i] < train_max):
            X_train_cut.append(X_train[i])
            y_train_cut.append(y_train[i])
            sc_train_cut.append(sc_train[i])
    
    X_test_cut = []
    y_test_cut = []
    sc_test_cut = []
    
    for i in range(len(X_test)):
        if (y_test[i] > train_min) and (y_test[i] < train_max):
            X_test_cut.append(X_test[i])
            y_test_cut.append(y_test[i])
            sc_test_cut.append(sc_test[i])
    
    # Builds training and testing arrays
    print("\nBuilding training and testing selected feature arrays...")
    X_train_select = Build_SelectFeatureArray(X_train_cut, feature_index_12feat)
    X_test_select  = Build_SelectFeatureArray(X_test_cut, feature_index_12feat)
    
    feature_coeff_label = feature_label_12feat.copy()
    feature_coeff_label.append("lr_intercept") # Adds field for linear regression y-intercept

    # Trains estimator
    print("\nTraining linear regression estimator...")
    lr_pipeline, lr_coeffs = Train_LinearRegression(
        X_train_select, 
        y_train_cut, 
        feature_coeff_label, 
        use_scaler = True)
    print(type(lr_pipeline))
    
    Write_MLWeights_ToCSV(
        output_csv_name + "_LR_Coeffs.csv",
        lr_coeffs, 
        feature_coeff_label
        )

    # Tests estimator and saves results
    output = "\nTesting " + str(len(feature_index_12feat)) + " features on " + str(int(train_min)) + "_" + str(int(train_max)) + " GeV..."
    print(output)

    csv_path = output_csv_name + "_Test_" + str(int(train_min)) + "_" + str(int(train_max)) + ".csv"

    TestAndSave_LinearRegression(
        feature_label_12feat,
        feature_index_12feat, 
        lr_pipeline, 
        lr_coeffs,
        X_test_select,
        y_test_cut, 
        sc_test_cut,
        train_min,
        train_max,   
        csv_path,
        use_scaler = True
        )

    print("Test and save complete!\n")



#################################################
#                                               #
#  TRAIN ML OVER 30 GeV BINS, USES 12 FEATURES  #
#                                               #
#################################################

# Builds outputs directories
output_directory_alt = output_directory + "Train_30GeV_Bins/"

try:
    os.mkdir(output_directory_alt)
    print("made directory")
except:
    print("directory already exists")
try:
    os.mkdir(output_directory_alt + "Plots_Actual/")
    os.mkdir(output_directory_alt + "Plots_Delta/")
    print("made subdirectories")
except:
    print("directory already exists")

training_bundle = [
    [10.,40.], [20.,50.], [30.,60.], [40.,70.],
    [50.,80.], [60.,90.]
    ]

# ONLY runs 12 features
for training_range in training_bundle:
    train_min = training_range[0]
    train_max = training_range[1]
    
    output_csv_name = output_directory_alt + "Train_" + train_bias + "_F12_" + str(int(train_min)) + "_" + str(int(train_max))
    
    X_train_cut = []
    y_train_cut = []
    sc_train_cut = []
    
    for i in range(len(X_train)):
        if (y_train[i] > train_min) and (y_train[i] < train_max):
            X_train_cut.append(X_train[i])
            y_train_cut.append(y_train[i])
            sc_train_cut.append(sc_train[i])
    
    X_test_cut = []
    y_test_cut = []
    sc_test_cut = []
    
    for i in range(len(X_test)):
        if (y_test[i] > train_min) and (y_test[i] < train_max):
            X_test_cut.append(X_test[i])
            y_test_cut.append(y_test[i])
            sc_test_cut.append(sc_test[i])
    
    # Builds training and testing arrays
    print("\nBuilding training and testing selected feature arrays...")
    X_train_select = Build_SelectFeatureArray(X_train_cut, feature_index_12feat)
    X_test_select  = Build_SelectFeatureArray(X_test_cut, feature_index_12feat)
    
    feature_coeff_label = feature_label_12feat.copy()
    feature_coeff_label.append("lr_intercept") # Adds field for linear regression y-intercept

    # Trains estimator
    print("\nTraining linear regression estimator...")
    lr_pipeline, lr_coeffs = Train_LinearRegression(
        X_train_select, 
        y_train_cut, 
        feature_coeff_label, 
        use_scaler = True)
    print(type(lr_pipeline))
    
    Write_MLWeights_ToCSV(
        output_csv_name + "_LR_Coeffs.csv",
        lr_coeffs, 
        feature_coeff_label
        )

    # Tests estimator and saves results
    output = "\nTesting " + str(len(feature_index_12feat)) + " features on " + str(int(train_min)) + "-" + str(int(train_max)) + " GeV..."
    print(output)

    csv_path = output_csv_name + "_Test_" + str(int(train_min)) + "_" + str(int(train_max)) + ".csv"

    TestAndSave_LinearRegression(
        feature_label_12feat,
        feature_index_12feat, 
        lr_pipeline, 
        lr_coeffs,
        X_test_select,
        y_test_cut, 
        sc_test_cut,
        train_min,
        train_max,   
        csv_path,
        use_scaler = True
        )

    print("Test and save complete!\n")

    
    
now = datetime.now()
dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
print("\nComplete!", dt_string)

made directory
made subdirectories

Building training and testing selected feature arrays...
Selecting data from master array...
Data ready. Feature array length: 661199 

Selecting data from master array...
Data ready. Feature array length: 661566 


Training linear regression estimator...

----- Fitting Linear Regression Estimator -----


Using StandardScaler. Data will be recentered and normalized.


Linear Regression Fit:
 Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])
36.84549162357447
[20.65562704]
[20.65562704 36.84549162]
Regression Coefficients:
jet_pt_raw 20.65562704422139
lr_intercept 36.84549162357447
<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.pipeline.Pipeline'>
ML weights .csv file closed.

Testing 1 features on 18-22 GeV...
<class 'sklearn.pipeline.Pipeline'>
ML results .csv file closed.
Test and save complete!


Testing 1 features on 28-32 GeV...
<class 'sklearn.pipeline.Pipeline'>
ML results .cs

ML results .csv file closed.
Test and save complete!


Building training and testing selected feature arrays...
Selecting data from master array...
Data ready. Feature array length: 135115 

Selecting data from master array...
Data ready. Feature array length: 135707 


Training linear regression estimator...

----- Fitting Linear Regression Estimator -----


Using StandardScaler. Data will be recentered and normalized.


Linear Regression Fit:
 Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])
39.1669057726291
[ 3.09747831e+00  1.45724742e+00 -9.88871117e-02  1.40152345e-01
 -2.61223991e+00 -9.50567821e-02  9.56286396e-02  7.34305163e-02
  4.22821085e-02  1.85568300e-02  1.51940867e-03  2.59350035e-01]
[ 3.09747831e+00  1.45724742e+00 -9.88871117e-02  1.40152345e-01
 -2.61223991e+00 -9.50567821e-02  9.56286396e-02  7.34305163e-02
  4.22821085e-02  1.85568300e-02  1.51940867e-03  2.59350035e-01
  3.91669058e+01]
Regression 


Linear Regression Fit:
 Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])
20.859096992793432
[ 5.91383303e+00  1.71452770e+00  3.55212629e+00 -4.50738306e-01
 -7.11192955e+00 -6.06281562e-01  3.50602866e-01  3.55801121e-01
  3.17507346e-01  1.08614172e-02 -6.84676272e-03 -1.74784756e-02]
[ 5.91383303e+00  1.71452770e+00  3.55212629e+00 -4.50738306e-01
 -7.11192955e+00 -6.06281562e-01  3.50602866e-01  3.55801121e-01
  3.17507346e-01  1.08614172e-02 -6.84676272e-03 -1.74784756e-02
  2.08590970e+01]
Regression Coefficients:
jet_pt_raw 5.913833031856506
jet_pt_corr 1.7145277026172108
jet_mass 3.552126286334988
jet_area -0.45073830640437423
jet_const_n -7.111929546205596
const_pt_mean -0.6062815619482498
const_1_pt 0.35060286634110355
const_2_pt 0.35580112148273735
const_3_pt 0.31750734637668093
const_4_pt 0.01086141723939105
jet_y -0.0068467627183275035
jet_rho -0.017478475553377743
lr_intercept 20.859096992793432
<class 'skle

ML results .csv file closed.
Test and save complete!


Complete! 2023/01/26 13:06:58
